# Azure completions の例

> 注意: openai ライブラリの新しいバージョンが利用可能です。https://github.com/openai/openai-python/discussions/742 を参照してください

この例では、Azure OpenAI サービスを使用した completions について説明します。

## セットアップ

まず、必要な依存関係をインストールします。

In [ ]:
! pip install "openai>=0.28.1,<1.0.0"

以下のセクションが正常に動作するように、まず最初にいくつかの設定を行う必要があります。`api_base`と`api_version`から始めましょう。`api_base`を見つけるには、https://portal.azure.com にアクセスし、リソースを見つけて、「リソース管理」→「キーとエンドポイント」の下にある「エンドポイント」の値を確認してください。

In [ ]:
import os
import openai

In [ ]:
openai.api_version = '2023-05-15'
openai.api_base = '' # Please add your endpoint here

次に、`api_type`と`api_key`を設定する必要があります。キーはポータルから取得するか、Microsoft Active Directory認証を通じて取得することができます。これに応じて、`api_type`は`azure`または`azure_ad`のいずれかになります。

### セットアップ: ポータル
まず、ポータルからキーを取得する方法を見てみましょう。https://portal.azure.com にアクセスし、リソースを見つけて、「リソース管理」→「キーとエンドポイント」の下で「キー」の値の一つを探してください。

In [ ]:
openai.api_type = 'azure'
openai.api_key = os.environ["OPENAI_API_KEY"]

> 注意: この例では、コード内で変数を設定することでライブラリがAzure APIを使用するように設定しました。開発時には、代わりに環境変数を設定することを検討してください：

```
OPENAI_API_BASE
OPENAI_API_KEY
OPENAI_API_TYPE
OPENAI_API_VERSION
```

### （オプション）セットアップ：Microsoft Active Directory認証
次に、Microsoft Active Directory認証を使用してキーを取得する方法を見てみましょう。ポータルからのキーの代わりにActive Directory認証を使用したい場合は、以下のコードのコメントアウトを解除してください。

In [ ]:
# from azure.identity import DefaultAzureCredential

# default_credential = DefaultAzureCredential()
# token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

# openai.api_type = 'azure_ad'
# openai.api_key = token.token

トークンは一定期間有効で、その後期限切れになります。すべてのリクエストで有効なトークンが送信されるようにするために、`requests.auth`にフックすることで期限切れのトークンを更新できます：

In [ ]:
import typing
import time
import requests
if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

session = requests.Session()
session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

openai.requestssession = session

## デプロイメント
このセクションでは、`gpt-3.5-turbo-instruct`モデルを使用してデプロイメントを作成し、それを使って補完を生成する方法について説明します。

### デプロイメント: 手動で作成する
ポータルでリソースの「リソース管理」->「モデル デプロイメント」に移動して、新しいデプロイメントを作成します。モデルとして`gpt-3.5-turbo-instruct`を選択してください。

In [ ]:
deployment_id = '' # Fill in the deployment id from the portal here

### 補完
それでは、デプロイメントにサンプルの補完を送信してみましょう。

In [ ]:
prompt = "The food was delicious and the waiter"
completion = openai.Completion.create(deployment_id=deployment_id,
                                     prompt=prompt, stop=".", temperature=0)
                                
print(f"{prompt}{completion['choices'][0]['text']}.")